## Building Context Aware Workflows with SingleStore

### Step 1: Install Required Packages & Dependencies

In [11]:
pip install openai langchain langchain-community langchain-openai singlestoredb --quiet

Note: you may need to restart the kernel to use updated packages.


### Step 2: Import Required Libraries and Initialize Components

In [10]:
from langchain_openai import OpenAIEmbeddings  # works after installing langchain-openai
from langchain_community.vectorstores import SingleStoreDB
from openai import OpenAI

### Step 3: Set Up SingleStore and OpenAI Credentials

In [12]:
SINGLESTORE_HOST = "Add host URL"   # your host
SINGLESTORE_USER = "admin"                     # your user
SINGLESTORE_PASSWORD = "Add your SingleStore DB password"    # your password
SINGLESTORE_DATABASE = "context_engineering"   # your database
OPENAI_API_KEY = "Add your OpenAI API key"

### Step 4: Connect to the SingleStore Database

In [13]:
connection_string = f"mysql://{SINGLESTORE_USER}:{SINGLESTORE_PASSWORD}@{SINGLESTORE_HOST}:3306/{SINGLESTORE_DATABASE}"

### Step 5: Initialize Embeddings and OpenAI Client

In [14]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

### Step 6: Initialize the SingleStore Vector Database

In [17]:
from langchain_community.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

vectorstore = SingleStoreDB(
    embedding=embeddings,
    table_name="context_memory",
    host=SINGLESTORE_HOST,
    user=SINGLESTORE_USER,
    password=SINGLESTORE_PASSWORD,
    database=SINGLESTORE_DATABASE,
    port=3306
)

### Step 7: Insert Knowledge into Long-Term Memory

In [18]:
docs = [
    {"id": "1", "text": "SingleStore unifies SQL and vector search in a single engine."},
    {"id": "2", "text": "Context engineering ensures AI agents always have the right context at the right time."},
    {"id": "3", "text": "SingleStore is ideal for real-time RAG pipelines due to low-latency queries."}
]

# Insert into vector DB
vectorstore.add_texts([d["text"] for d in docs], ids=[d["id"] for d in docs])
print("✅ Knowledge inserted into SingleStore")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ Knowledge inserted into SingleStore


### Step 8: Retrieve Relevant Context

In [19]:
query = "Why is SingleStore useful for context engineering?"
results = vectorstore.similarity_search(query, k=2)

print("🔹 Retrieved Context:")
for r in results:
    print("-", r.page_content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🔹 Retrieved Context:
- SingleStore is ideal for real-time RAG pipelines due to low-latency queries.
- SingleStore unifies SQL and vector search in a single engine.


### Step 9: Build Prompt for LLM

In [20]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

user_input = "Explain context engineering using SingleStore."

context = "\n".join([r.page_content for r in results])

prompt = f"""
You are a helpful AI agent.
User asked: {user_input}
Relevant context from memory:
{context}
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print("🔹 Agent Answer:\n", response.choices[0].message.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🔹 Agent Answer:
 Context engineering in the realm of artificial intelligence and data management involves preparing and optimizing data for use in context-aware applications, such as real-time retrieval-augmented generation (RAG) systems. When leveraging SingleStore for this purpose, it's essential to highlight key features that make it a powerful tool.

1. **Low-Latency Queries**: SingleStore is optimized for real-time data processing, which is crucial for RAG applications that rely on immediate access to relevant context. This allows the system to retrieve and analyze data quickly, enabling rapid response times and enhancing user experience.

2. **Unified SQL and Vector Search**: One of SingleStore's standout features is its ability to handle both SQL queries and vector searches within the same engine. This unification simplifies the data workflow, allowing developers to seamlessly integrate structured data (managed through SQL) with unstructured data (accessed via vector search). 



### Step 10: Store Conversation Back (Short-Term → Long-Term Memory)

In [21]:
vectorstore.add_texts([
    f"User: {user_input}", 
    f"Assistant: {response.choices[0].message.content}"
])

print("✅ Conversation stored back into SingleStore for future retrieval")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ Conversation stored back into SingleStore for future retrieval


### Step 11: Test Retrieval Again

In [22]:
followup_query = "What did we discuss earlier about context engineering?"
followup_results = vectorstore.similarity_search(followup_query, k=3)

print("🔹 Follow-up Retrieved Context:")
for r in followup_results:
    print("-", r.page_content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🔹 Follow-up Retrieved Context:
- User: Explain context engineering using SingleStore.
- Context engineering ensures AI agents always have the right context at the right time.
- Assistant: Context engineering in the realm of artificial intelligence and data management involves preparing and optimizing data for use in context-aware applications, such as real-time retrieval-augmented generation (RAG) systems. When leveraging SingleStore for this purpose, it's essential to highlight key features that make it a powerful tool.

1. **Low-Latency Queries**: SingleStore is optimized for real-time data processing, which is crucial for RAG applications that rely on immediate access to relevant context. This allows the system to retrieve and analyze data quickly, enabling rapid response times and enhancing user experience.

2. **Unified SQL and Vector Search**: One of SingleStore's standout features is its ability to handle both SQL queries and vector searches within the same engine. This unificat